This is a classification problem. You should have mentioned classification-appropriate models and metrics.
 • You can propose different models for different types of data.
 • I was also looking for hyperparameter tuning options, including cross-validation approaches.
 • And a mention of how you’ll assess whether your model is biased.
 • And interpretability, which is also a requirement of the project